<a href="https://colab.research.google.com/github/fiorentin0/deberes/blob/main/_hw6_llm_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Программистская часть (4 балла)

## Natural Language Inference (2 балла)

Natural Language Inference — это задача "понимания" отношений между двумя утверждениями. Одно может следовать из другого (если (1) "все кошки полосатые" (2) "моя кошка полосатая", то из 1 следует 2 — но не наоборот), может противоречить ("все кошки полосатые" и "моя кошка не полосатая") или относиться нейтрально ("все кошки полосатые" и "моя кошка любит играть с клубком")

Заведите модель, проверьте разные пары утверждений. Будет здорово, если получится найти что-то, на чем она отрабатывает неправильно — но это не обязательное требование для ДЗ.

In [ ]:
# по техническим причинам, укажем AutoModelForSequenceClassification вместо просто AutoModel

from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "cointegrated/rubert-base-cased-nli-threeway"  # русскоязычная модель для этой задачи
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

text1 = "Сократ - человек, а все люди смертны."
text2 = "Сократ никогда не умрёт."

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/545 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.62M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/711M [00:00<?, ?B/s]

Вызываем модель вот так. `return_tensors='pt'` говорит вернуть не список токенов, а так называемый тип `tensor`. Это что-то вроде `np.array`, но используется в нейросетях.

In [ ]:
out = model(**tokenizer(text1, text2, return_tensors='pt'))

Посмотрите, что лежит в переменной out — результате вызова модели от токенов предложений

In [ ]:
out

SequenceClassifierOutput(loss=None, logits=tensor([[-2.2051,  2.3795, -0.4113]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

`logits` — это то, что на самом деле отдают все нейросети. Их количество соответствует количеству классов. У нас три класса и один текст, поэтому мы получили матрицу размера одна строка на три числа. **Обратите внимание:** несмотря на то, что у нас одна пара текстов, мы всё равно получили *массив массивов*.

Логиты пропоцориональны вероятностям: чем больше значение, тем более вероятен класс.

Здесь полезно вспомнить логистическую регрессию: у нас есть число, а мы хотим его преобразовать его в диапазон от 0 до 1 — вероятность. Только теперь у нас целый массив чисел, и надо, чтобы они еще и суммировались к 1 (сумма вероятностей разных классов). Это делается называется **softmax** и делается так:

$s(x_i) = \frac{e^{x_i}}{\sum_{j=1}^{n} e^{x_j}}$

**Реализуйте функцию softmax**

In [ ]:
out.logits

tensor([[-2.2051,  2.3795, -0.4113]], grad_fn=<AddmmBackward0>)

In [ ]:
import numpy as np

def softmax(logits):
    # You may want to convert logits to a NumPy array
    np_logits = logits.detach().numpy()
    # Compute the exponential of each logit
    exp_logits = np.exp(np_logits)
    # Compute the sum of exponentials for all logits
    sum_logits = np.sum(exp_logits)
    # Calculate softmax for each logit
    softmax_scores = exp_logits/sum_logits

    return softmax_scores

softmax(out.logits)

array([[0.00952592, 0.93320656, 0.05726758]], dtype=float32)

Сверьте вашу реализацию с готовой:

In [ ]:
import torch

proba = torch.softmax(out.logits, -1)
proba

tensor([[0.0095, 0.9332, 0.0573]], grad_fn=<SoftmaxBackward0>)

In [ ]:
# вот так можно получить список, а не tensor, если пользуетесь готовой реализацией
proba[0].tolist()

[0.009525920264422894, 0.9332064986228943, 0.057267576456069946]

У модели также параметр-словарь: из порядкового номера в "словесный" лейбл

In [ ]:
model.config.id2label

{0: 'entailment', 1: 'contradiction', 2: 'neutral'}

Используйте словарь с названиями лейблов и посчитанные вероятности, чтобы получить словарь вида `{лейбл: вероятность}`

In [ ]:
labels_proba = {}
for i, j in model.config.id2label.items():
  labels_proba[j] = proba[0].tolist()[i]
labels_proba

{'entailment': 0.009525920264422894,
 'contradiction': 0.9332064986228943,
 'neutral': 0.057267576456069946}

Самый вероятный класс для примера про Сократа был, наверное, ожидаем. Получилось ли у вас найти что-то, с чем модель не справилась?
**contradiction/противоречие:** [ваш ответ]

## Анонимизация (2 балла)

Мы научились вызывать NER-тэггер. Воспользуйтесь семинаром, чтобы написать функцию, которая принимает некоторое предложение, а возвращает его же с удаленными named entity, замененными на их тэг. Например, `"Меня зовут Лена, я живу в Долгопрудном, работаю в Яндексе и преподаю в Вышке"` —>  `"Меня зовут <PER>, я живу в <LOC>, работаю в <LOC> и преподаю в <LOC>"`

In [ ]:
# YOUR CODE HERE
from transformers import pipeline
classifier = pipeline("ner", model="Babelscape/wikineural-multilingual-ner", grouped_entities=True)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/token_classification.py:168: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="simple"` instead.
  warnings.warn(


In [ ]:
def replace_to_ner_tags(unput_str):
  tagged_text = classifier(unput_str)

  len_tags = len(tagged_text)
  idx = 0
  end = len(tagged_text)
  while idx < end:
    ner = dict(tagged_text[idx])
    unput_str = unput_str.replace(ner["word"], f'<{ner["entity_group"]}>', 1)
    idx+=1

  return unput_str

text = "Меня зовут Лена, я живу в Долгопрудном, работаю в Яндексе и преподаю в Вышке"
replace_to_ner_tags(text)

'Меня зовут <PER>, я живу в <LOC>, работаю в <LOC> и преподаю в <LOC>'

# Прикольная часть (3 балла)

Вы можете использовать BLOOM API - [bigscience/bloom](https://huggingface.co/bigscience/bloom) (окошко справа; нужно зарегистрироваться и залогиниться)

Для каждой задачи предоставьте краткое __описание__, как вы пробовали ее решить, и __скриншот__ из API.

__Например:__ Tony is talking to Darth Vader ([BLOOM API](https://huggingface.co/bigscience/bloom)). Черный текст — составленный промпт, синий — сгенерированный.

![img](https://i.imgur.com/a1QhKF7.png)

Ничего страшного, если вам придется несколько раз перезапустить генерацию. Но если вам приходится делать это больше 1-2 раз или у вас чаще получается ерунда, чем то, что хочется — имеет смысл попробовать переформулировать промпт.

## Диалог (1 балл)

**0.5 балла:** сгенерируйте диалог между любыми двумя персонажами из списка:
* любая знаменитость или историческая фигура
* любой персонаж книг/игр/другого медиа
* любое существо из мифологии или фольклора
* *вы*

![img](https://i.imgur.com/gA4If6R.png)

**0.5 балла:** то же самое (можно взять других персонажей, можно тех же), но теперь с дополнительной информацией о них или теме их разговора

![img](https://i.imgur.com/qbbCi1i.png)

## Перевод (1 балл)

### ...слов

**0.25 балла:** составьте запрос к модели, переводящий любую строку любого стихтворения на другой язык.
![img](https://i.imgur.com/iTXVtS2.png)

**0.25 балла:** то же самое, но никак не формулируйте задачу, а только предоставьте несколько примеров (few-shot prompting)
![img](https://i.imgur.com/iIZ9A2m.png)

### ...величин
**0.25 балла:** составьте запрос к модели, переводящий все метрические единицы в какой-либо фразе в имперские или наоборот. Помните, что 1 километр и 1 миля не равны.

![img](https://i.imgur.com/9ZsSbVF.png)

**0.25 балла:** то же самое, но никак не формулируйте задачу, а только предоставьте несколько примеров (few-shot prompting)

![img](https://i.imgur.com/2RnMZ52.png)

## Ваш опыт (1 балл)

**0.5 балла**

Покажите пример самого интересного (на ваш взгляд) успешного взаимодействия с любой чат-моделью, которой вы пользовались. Возможно, модель помогала вам написать официально-деловое письмо на иностранном языке или придумать сюжет для фанфика?

Подойдет ChatGPT, Gemini, Bard или даже любые боты в телеграме. А если еще не пользовались — самое время попробовать!

**0.5 балла**

То же самое, но неудачное взаимодействие. С какой задачей модель так и не справилась, несмотря на ваши попытки?

# Ещё 5 баллов

Задача - предсказать POS-теги для каждого слова в тексте на основе эмбеддингов BERT.

Мы будем работать с данными в формате [Universal Dependencies](https://github.com/UniversalDependencies). Вы можете выбрать любой язык, для которого есть файлы train и dev/test.

In [ ]:
!git clone https://github.com/UniversalDependencies/UD_Russian-Taiga.git

Cloning into 'UD_Russian-Taiga'...
remote: Enumerating objects: 241, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 241 (delta 25), reused 35 (delta 13), pack-reused 194
Receiving objects: 100% (241/241), 12.70 MiB | 15.19 MiB/s, done.
Resolving deltas: 100% (147/147), done.


In [ ]:
train_filename = "/content/UD_Russian-Taiga/ru_taiga-ud-train.conllu"
test_filename = "/content/UD_Russian-Taiga/ru_taiga-ud-test.conllu"

Мы будем работать с форматом conll-U, в котором хранятся UD данные. Для обработки таких данных уже существуют готовые библиотеки. Нам нужно извлечь токены и pos-тэги. Напишем функцию:

In [ ]:
!pip install conllu

In [ ]:
import conllu
from tqdm.auto import tqdm

def get_dataset(filename):
    with open(filename) as f:
        data = f.read()

    text = []  # здесь мы будем хранить токены
    labels = []  # здесь мы будем хранить pos-тэги
    for sent in conllu.parse(data):
        tokens, pos = list(zip(*[(token["form"], token["upos"]) for token in sent if token["upos"] != "_"]))
        text.append(" ".join(tokens))  # нам важен контекст предложения
        labels.extend(pos)
    return text, labels

In [ ]:
X_train, y_train = get_dataset(train_filename)
X_test, y_test = get_dataset(test_filename)

Теперь нам нужно получить эмбеддинги из модели. Мы будем работать с [мультилингвальным BERT](https://huggingface.co/google-bert/bert-base-multilingual-cased):

In [ ]:
from transformers import BertTokenizer, BertModel

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertModel.from_pretrained("bert-base-multilingual-cased")
# model = model.to("cuda:0") # перенесем на GPU, !!не запустить на маке с М1/М2!!

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Для начала напишем функцию, которая достает нам эмбеддинг предложения из модели:

In [ ]:
#def get_embedding(sentence, device="cuda:0"):
def get_embedding(sentence):
    inputs = tokenizer(sentence, padding=True, truncation=True,
                    return_tensors="pt").input_ids
    # inputs = inputs.to(device)
    outputs = model(inputs)
    outputs = outputs["last_hidden_state"].detach().cpu().numpy().squeeze(0) # берем только последний слой
    return outputs

In [ ]:
get_embedding('some random text')

array([[ 0.33513942, -0.3053692 ,  0.02044745, ...,  0.69708145,
         0.25087476, -0.10693159],
       [ 0.15359867, -0.6349667 , -0.09096607, ...,  1.1083326 ,
         0.47730675,  0.01249174],
       [ 0.29667854, -0.42799792, -0.3325467 , ...,  0.9175281 ,
         0.5084595 , -0.18118727],
       [ 0.31915006, -0.48856074, -0.00151148, ...,  0.85804194,
        -0.09692569, -0.04755181],
       [ 0.17154461, -0.58457065,  0.19676194, ...,  0.7727915 ,
         0.23590934,  0.05959709]], dtype=float32)

Как вы помните, BERT оперирует не на уровне слов, а на уровне word pieces, т. е. частей слов. Чтобы получить эмбеддинг токена, нам нужно объединить части слов. Нам нужны индексы токенов, которые принадлежат к одному и тому же токену. Мы должны получить список списков

In [ ]:
def decode_tokens(sentence):
    tokens = []
    idx = 1

    for token in sentence.split(' '):
        encoded_word = tokenizer(token, add_special_tokens=False).input_ids  # токенизуруем один токен и получим части одного токена, подсказка: добавьте add_special_tokens=False
        tokens.append(encoded_word) # напишите цикл, который возвращает список, где каждый токен представлен как список индексов его подчастей
        idx += 1
    return tokens

decode_tokens(X_test[0])

[[60682],
 [548, 83969],
 [543],
 [27446, 10353],
 [513, 80107],
 [549],
 [520, 80107],
 [11582],
 [13106],
 [10757],
 [95152],
 [549],
 [25330, 10179],
 [44711],
 [109300, 25219, 72226],
 [51290],
 [119]]

In [ ]:
X_test[0]

'Ещё зимой в армиях ДНР и ЛНР было более 500 танков и тысяча боевых бронированных машин .'

In [ ]:
# проверим, получился ли правильный список
assert len(decode_tokens(X_test[0])) == len(X_test[0].split())

Теперь напишем функцию, которая получает наш список списков с индексами и выдает **усредненный** эмбеддинг токена

In [ ]:
import numpy as np

def get_embedding_per_token(tokens, sentence_embedding, length=768):
    embeddings = np.zeros((len(tokens), length))  # создадим пустую матрицу размеров токены х размер эмбеддингов
    for num, token in enumerate(tokens):
        embeddings[num,:] = np.average(sentence_embedding[num]) # возьмите срез эмбеддинга предложения, равный нужному слову, и усредните
        print(num, token, sentence_embedding[num])
    return embeddings

sentence_embedding = get_embedding(X_test[0])
tokens = decode_tokens(X_test[0])

sentence_embedding[0][tokens[0]]

#x = get_embedding_per_token(tokens, sentence_embedding)

IndexError: index 60682 is out of bounds for axis 0 with size 768

Теперь давайте объединим все функции и напишем функцию, которая обрабатывает все предложения

In [ ]:
def encode_dataset(X):
    embeddings = []
    for sentence in tqdm(X):
        sentence_embedding = ...
        tokens = ...
        token_embeddings = ...
        embeddings.append(token_embeddings)
    return np.concatenate(embeddings)  # возвращаем матрицу

In [ ]:
transformed_X_train = ...
transformed_X_test = ...

  0%|          | 0/13306 [00:00<?, ?it/s]

  0%|          | 0/1659 [00:00<?, ?it/s]

In [ ]:
assert len(transformed_X_train) == len(y_train)
assert len(transformed_X_test) == len(y_test)

Теперь давайте обучим логистичесчкую регрессию на основе полученных эмбеддингов

In [ ]:
from sklearn.linear_model import LogisticRegression

#logreg = ...
# обучите модель

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

Теперь давайте измерим получившееся качество

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, classification_report

# y_pred = ...
print("Accuracy is ", accuracy_score(y_test, y_pred))
print("F1-score is ", f1_score(y_test, y_pred, average="weighted"))

print(classification_report(y_test, y_pred))

Прокомментируйте получившиеся результаты

**Ваш ответ**: